In [4]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

## Create a Basic Model of the Form y = mx + c

In [25]:
# Create a simple Keras model.
x = np.arange(1,11)*2 - 2*5
y = x*2 - 1 + np.random.randn(10)*0.1

print(x, y)

[-8 -6 -4 -2  0  2  4  6  8 10] [-17.04088579 -12.95622548  -8.9484593   -4.85814225  -0.8697331
   2.84736949   7.06092187  11.06872737  14.99017446  19.16439948]


In [26]:
dataset = tf.data.Dataset.from_tensor_slices(([[val] for val in x], y))

In [27]:
dataset = dataset.batch(2)

In [28]:
[*dataset.take(1)]

[(<tf.Tensor: shape=(2, 1), dtype=int32, numpy=
  array([[-8],
         [-6]], dtype=int32)>,
  <tf.Tensor: shape=(2,), dtype=float64, numpy=array([-17.04088579, -12.95622548])>)]

In [31]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=1, input_shape=[1]),
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='mean_absolute_error')

model.fit(dataset, epochs=500, callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, monitor='loss')])

Epoch 1/500
5/5 [==============================] - 0s 7ms/step - loss: 7.0294
Epoch 2/500
5/5 [==============================] - 0s 8ms/step - loss: 6.8026
Epoch 3/500
5/5 [==============================] - 0s 9ms/step - loss: 6.5822
Epoch 4/500
5/5 [==============================] - 0s 8ms/step - loss: 6.3624
Epoch 5/500
5/5 [==============================] - 0s 6ms/step - loss: 6.1430
Epoch 6/500
5/5 [==============================] - 0s 8ms/step - loss: 5.9238
Epoch 7/500
5/5 [==============================] - 0s 6ms/step - loss: 5.7048
Epoch 8/500
5/5 [==============================] - 0s 7ms/step - loss: 5.4860
Epoch 9/500
5/5 [==============================] - 0s 7ms/step - loss: 5.2673
Epoch 10/500
5/5 [==============================] - 0s 7ms/step - loss: 5.0486
Epoch 11/500
5/5 [==============================] - 0s 6ms/step - loss: 4.8301
Epoch 12/500
5/5 [==============================] - 0s 7ms/step - loss: 4.6117
Epoch 13/500
5/5 [==============================] - 0s 7ms/st

## Generate a SavedModel

In [33]:
output_path = './model/model_v1'
Path(output_path).mkdir(exist_ok=True, parents=True)

In [34]:
model.save(output_path)

INFO:tensorflow:Assets written to: ./model/model_v1/assets


## Convert the SavedModel to TFLite

In [35]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(output_path)
tflite_model = converter.convert()

In [36]:
tflite_model_file = Path('/model/model_v1.tflite')
tflite_model_file.write_bytes(tflite_model)

916

## Initialize the TFLite Interpreter To Try It Out

[{'name': 'serving_default_dense_8_input:0',
  'index': 0,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [37]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [44]:
input_details

[{'name': 'serving_default_dense_8_input:0',
  'index': 0,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [45]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 3,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [48]:
test_in = np.array([[5.]], dtype=np.float32)
in_index = input_details[0]['index']
out_index = output_details[0]['index']


interpreter.set_tensor(in_index, test_in)
interpreter.invoke()
test_out = interpreter.get_tensor(out_index)
print(test_in, test_out)


[[5.]] [[9.057872]]
